In [2]:
import pandas as pd
import seaborn as sns

In [3]:
rdata = pd.read_csv('../data/s.csv')

as1 = rdata.Refugees != '*'
as2 = rdata.AsylumSeekers != '*'
rdata = rdata[as1 & as2]

rdata['Refugees'] = rdata['Refugees'].astype(float)
rdata['AsylumSeekers'] = rdata['AsylumSeekers'].astype(float)

# pad with 0s
rdata = rdata.fillna(0)
rdata['T'] = rdata['Refugees'] + rdata['AsylumSeekers']

# only use subset
rdata = rdata[['Destination', 'T', 'Year']]

In [8]:
d = set(rdata.Destination)
y = set(rdata.Year)
p = pd.MultiIndex.from_product([d,y], names=['destinations', 'year'])
rdata.reindex(p)
rdata.head(5)

,Destination,T,Year
0,Lebanon,1200,1968
1,Lebanon,1200,1969
2,Lebanon,500,1979
3,Italy,2,1980
4,Lebanon,500,1980


In [48]:
pop = pd.read_csv('../data/demographics-wb.csv')
pop = pop[['Country Name',  '2011', '2012', '2013','2014']]
pop.columns = ['Destination', '2011_pop', '2012_pop', '2013_pop', '2014_pop']

,Destination,2011_pop,2012_pop,2013_pop,2014_pop
0,Aruba,101932,102384,102911,103431
1,Andorra,77865,78360,79218,80153
2,Afghanistan,29105480,29824536,30551674,31280518
3,Angola,20180490,20820525,21471618,22137261
4,Albania,2904780,2900489,2897366,2894475


In [5]:
# Find top10
r = rdata
r = r.drop(['Year'], 1)
g = r.groupby('Destination').sum()
top10 = g.sort('T', ascending=False).head(10)
top10

,T
Destination,
Turkey,2393091
Lebanon,2134927
Jordan,829210
Iraq,528157
Egypt,283099
Germany,256993
Sweden,137136
Netherlands,51502
Libya,39428


r = rdata[rdata['Year'] > 2009]
r = r.drop(['Year'], 1)
g = r.groupby('Destination').sum()
top10 = g.sort('T', ascending=False).head(10)
top10

In [61]:
rd = rdata[rdata['Year'] > 2010]
d_pivot = pd.pivot_table(rd, columns=['Destination', 'Year'], values=['T'])
df = d_pivot.unstack()
# to output:
df.reset_index().to_json(orient='records', path_or_buf='../app/src/data/stats.json')

In [62]:
rfs = df.reset_index()
rfs = rfs.merge(pop, on='Destination', how='left')
rfs.to_json(orient='records', path_or_buf='../app/src/data/stats.json')

In [66]:
rfs.head(10)
rfs['T'] = rfs[2011] + rfs[2012] + rfs[2013] + rfs[2014]

rfs['rate_avg'] = rfs['T'] / rfs['2014_pop']
rfs.sort('T', ascending=0).to_json(orient='records', path_or_buf='../app/src/data/stats.json')